# Defining a Fixed-Floating Interest Rate Swap

Simple introduction to the inputs required to define a Ibor deposit and a FRA

In [1]:
from financepy.finutils import *
from financepy.products.rates import *

####################################################################
# FINANCEPY BETA Version 0.191 - This build:  17 Jan 2021 at 21:17 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Creating the Swap

We begin by setting the market conventions we wish to use.

In [2]:
swapCalendarType = CalendarTypes.TARGET
bus_day_adjust_type = BusDayAdjustTypes.FOLLOWING
date_gen_rule_type = DateGenRuleTypes.BACKWARD

We then define the Fixed Leg

In [3]:
fixed_coupon = 0.05
fixedFreqType = FrequencyTypes.ANNUAL
fixed_day_count_type = DayCountTypes.ACT_360

And then we define the Floating Leg

In [4]:
float_spread = 0.0
floatFreqType = FrequencyTypes.SEMI_ANNUAL
float_day_count_type = DayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [5]:
swapType = FinSwapTypes.RECEIVE
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

In [6]:
start_date = FinDate(20, 6, 2018)
maturity_date = FinDate(20, 9, 2025)

We can now create the swap object.

In [7]:
swap = FinIborSwap(start_date,
                    maturity_date,
                    swapType,
                    fixed_coupon,
                    fixedFreqType,
                    fixed_day_count_type,
                    notional,
                    float_spread,
                    floatFreqType,
                    float_day_count_type,
                    swapCalendarType,
                    bus_day_adjust_type,
                    date_gen_rule_type)

In [8]:
print(swap)

OBJECT TYPE: FinIborSwap
OBJECT TYPE: FinFixedLeg
START DATE: 20-JUN-2018
TERMINATION DATE: 20-SEP-2025
MATURITY DATE: 22-SEP-2025
NOTIONAL: 10000000
PRINCIPAL: 0.0
LEG TYPE: FinSwapTypes.RECEIVE
COUPON: 0.05
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.TARGET
BUS DAY ADJUST: BusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: DateGenRuleTypes.BACKWARD

OBJECT TYPE: FinFloatLeg
START DATE: 20-JUN-2018
TERMINATION DATE: 20-SEP-2025
MATURITY DATE: 22-SEP-2025
NOTIONAL: 10000000
SWAP TYPE: FinSwapTypes.PAY
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.TARGET
BUS DAY ADJUST: BusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: DateGenRuleTypes.BACKWARD



In [9]:
swap.print_flows()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END      DAYS  YEARFRAC    RATE      PAYMENT 
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556  5.000000    127777.78
20-SEP-2019  20-SEP-2018  20-SEP-2019   365  1.013889  5.000000    506944.44
21-SEP-2020  20-SEP-2019  21-SEP-2020   367  1.019444  5.000000    509722.22
20-SEP-2021  21-SEP-2020  20-SEP-2021   364  1.011111  5.000000    505555.56
20-SEP-2022  20-SEP-2021  20-SEP-2022   365  1.013889  5.000000    506944.44
20-SEP-2023  20-SEP-2022  20-SEP-2023   365  1.013889  5.000000    506944.44
20-SEP-2024  20-SEP-2023  20-SEP-2024   366  1.016667  5.000000    508333.33
22-SEP-2025  20-SEP-2024  22-SEP-2025   367  1.019444  5.000000    509722.22
START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (bp): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END 

## Valuation

In [10]:
valuation_date = FinDate(20,3,2020)

In [11]:
settlement_date = valuation_date.addWeekDays(0)

This is just using a flat curve without any Ibor accrual factors.

In [12]:
from financepy.market.curves.FinDiscountCurveFlat import FinDiscountCurveFlat

In [13]:
discount_curve = FinDiscountCurveFlat(valuation_date, 0.05, FrequencyTypes.SEMI_ANNUAL)

Need to specify the next fixing on the floating leg

In [14]:
swap.value(settlement_date, discount_curve, discount_curve)

254993.1399350157

We can examine the individual legs - here is the floating leg valuation

In [15]:
swap.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    IBOR      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556    0.00000         0.00  0.00000000         0.00         0.00
20-MAR-2019  20-SEP-2018  20-MAR-2019   181  0.502778    0.00000         0.00  0.00000000         0.00         0.00
20-SEP-2019  20-MAR-2019  20-SEP-2019   184  0.511111    0.00000         0.00  0.00000000         0.00         0.00
20-MAR-2020  20-SEP-2019  20-MAR-2020   182  0.505556    0.00000         0.00  0.00000000         0.00         0.00
21-SEP-2020  20-MAR-2020  21-SEP-2020   185  0.513889    4.91870    252766.49  0.97534651    246534.91    246534.91
22-MAR-2021  21-SEP-2020  22-MAR-2021   182  0.505556    4.92370    248920.14  0.95165783    236886.80    483421.71
20-SEP-2021  22-MAR-2021  20-SEP-2021   182  0.505556 

And here is the fixed leg valuation

In [16]:
swap.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556   5.00000    127777.78  0.00000000         0.00         0.00
20-SEP-2019  20-SEP-2018  20-SEP-2019   365  1.013889   5.00000    506944.44  0.00000000         0.00         0.00
21-SEP-2020  20-SEP-2019  21-SEP-2020   367  1.019444   5.00000    509722.22  0.97534651    497155.79    497155.79
20-SEP-2021  21-SEP-2020  20-SEP-2021   364  1.011111   5.00000    505555.56  0.92850948    469413.12    966568.91
20-SEP-2022  20-SEP-2021  20-SEP-2022   365  1.013889   5.00000    506944.44  0.88376869    448021.63   1414590.54
20-SEP-2023  20-SEP-2022  20-SEP-2023   365  1.013889   5.00000    506944.44  0.84118376    426433.43   1841023.97
20-SEP-2024  20-SEP-2023  20-SEP-2024   366  1.016667   5.00000    

In [17]:
swap.pv01(valuation_date, discount_curve)

5.272690754192439

In [19]:
swap.swap_rate(valuation_date, discount_curve)

0.04516388971357023

## Including a final exchange of principal

In [27]:
swap.value(settlement_date, discount_curve, discount_curve)

254993.1399350157

In [28]:
swap.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    IBOR      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556    0.00000         0.00  0.00000000         0.00         0.00
20-MAR-2019  20-SEP-2018  20-MAR-2019   181  0.502778    0.00000         0.00  0.00000000         0.00         0.00
20-SEP-2019  20-MAR-2019  20-SEP-2019   184  0.511111    0.00000         0.00  0.00000000         0.00         0.00
20-MAR-2020  20-SEP-2019  20-MAR-2020   182  0.505556    0.00000         0.00  0.00000000         0.00         0.00
21-SEP-2020  20-MAR-2020  21-SEP-2020   185  0.513889    4.91870    252766.49  0.97534651    246534.91    246534.91
22-MAR-2021  21-SEP-2020  22-MAR-2021   182  0.505556    4.92370    248920.14  0.95165783    236886.80    483421.71
20-SEP-2021  22-MAR-2021  20-SEP-2021   182  0.505556 

As expected we see the final PV of the floating Leg is par (same index and discounting curve).

In [29]:
swap.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556   5.00000    127777.78  0.00000000         0.00         0.00
20-SEP-2019  20-SEP-2018  20-SEP-2019   365  1.013889   5.00000    506944.44  0.00000000         0.00         0.00
21-SEP-2020  20-SEP-2019  21-SEP-2020   367  1.019444   5.00000    509722.22  0.97534651    497155.79    497155.79
20-SEP-2021  21-SEP-2020  20-SEP-2021   364  1.011111   5.00000    505555.56  0.92850948    469413.12    966568.91
20-SEP-2022  20-SEP-2021  20-SEP-2022   365  1.013889   5.00000    506944.44  0.88376869    448021.63   1414590.54
20-SEP-2023  20-SEP-2022  20-SEP-2023   365  1.013889   5.00000    506944.44  0.84118376    426433.43   1841023.97
20-SEP-2024  20-SEP-2023  20-SEP-2024   366  1.016667   5.00000    

In [33]:
swap._fixed_leg.value(settlement_date, discount_curve)

2636345.37709622

In [35]:
swap._floatLeg.value(settlement_date, discount_curve, discount_curve)

-2381352.237161204

Copyright (c) 2020 Dominic O'Kane